In [5]:
import os
from glob import glob
from converter import Converter
import pandas as pd

In [2]:
raw_dir = '../../raw_videos/google-drive'
repaired_dir = os.path.join(raw_dir,'repaired')

# Detect and decide

In [15]:
input_files = sorted(glob(os.path.join(raw_dir,'*.avi')))

repair_info = {}

conv = Converter()
for f1 in input_files:
    fn,ext = os.path.splitext(os.path.basename(f1))
    if type(conv.probe(f1).video.bitrate)==type(None):
        f2 = os.path.join(repaired_dir,fn+'-repaired'+ext)
        repair_info[fn] = 'repaired' if os.path.exists(f2) else 'corrupt'
    else:
        repair_info[fn] = 'valid'

In [18]:
df = pd.DataFrame(repair_info.items(),columns=['file','status'])

writer = pd.ExcelWriter('repair-info.xlsx')
df.to_excel(writer,sheet_name='sheet1',index=None)
sheet = writer.sheets['sheet1']
for col,width in zip('AB',[60,10]):
    sheet.column_dimensions[col].width = width
writer.save()

# Repair

In [22]:
input_files = sorted(glob(os.path.join(raw_dir,'*.avi')))

conv = Converter()
for f1 in input_files:
    if type(conv.probe(f1).video.bitrate)==type(None):
        fn,ext = os.path.splitext(os.path.basename(f1))
        f2 = os.path.join(repaired_dir,fn+'-repaired'+ext)
        if not os.path.exists(f2):
#             print(fn)
            convert = conv.convert(f1, f2, {
                'format':'avi', 
                'video': {'codec':'copy'},
                })
            for progress in convert:
                print(f'{fn}   {100*progress:.1f}%',end='\r')
            print()

Pa_Sun_14dpf_GroupB_n2c_2020-06-21-114550-0000   90.0%
